# Dataset adjustments

In [4]:
import pandas as pd 

In [5]:
#storing the datasets as pandas dataframes

train_data = pd.read_csv('../Data/Processed Dataset TF-IDF Train')
test_data = pd.read_csv('../Data/Processed Dataset TF-IDF Test')

In [6]:
#removing unnecessary columns

train_data = train_data.drop('time of day', axis = 1)
train_data = train_data.drop('id', axis = 1)
train_data = train_data.drop('date_x', axis = 1)
train_data = train_data.drop('user', axis = 1)

test_data = test_data.drop('time of day', axis = 1)
test_data = test_data.drop('id', axis = 1)
test_data = test_data.drop('date_x', axis = 1)
test_data = test_data.drop('user', axis = 1)

In [7]:
train_data = train_data.drop('rendered_content', axis = 1)

In [8]:
train_data = train_data.drop('Adjusted Tweet', axis = 1)

In [9]:
test_data = test_data.drop('rendered_content', axis = 1)

In [10]:
test_data = test_data.drop('Adjusted Tweet', axis = 1)

In [11]:
train_data

,hour of tweet,Early Morning Count,Morning Count,Noon Count,Eve Count,Night Count,Late Night Count,user_followers,Months Since Creation of Account,Negative Score,...,youre,youtube,yr,zaporizhzhia,zaporozhye,zelenskiy,zelensky,zelenskys,zelenskyy,zone
0,23,0,0,0,0,1,0,795,145,0.077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,0,0,0,0,1,0,2506,81,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23,0,0,0,0,1,0,2659,167,0.107,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23,0,0,0,0,1,0,394,10,0.081,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22,0,0,0,0,1,0,113,113,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46284,23,0,0,0,0,1,0,200,161,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46285,23,0,0,0,0,1,0,7078,92,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46286,22,0,0,0,0,1,0,3554,145,0.184,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46287,12,0,0,1,0,0,0,190,6,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
#installing classification and regresion model library 
pip install lightgbm

SyntaxError: invalid syntax (391950818.py, line 2)

In [13]:
from lightgbm import LGBMClassifier, LGBMRegressor

# Hurdle Regression class

In [37]:
from typing import Optional, Union
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.utils.estimator_checks import check_estimator
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

class HurdleRegression(BaseEstimator):
    """ Regression model which handles excessive zeros by fitting a two-part model and combining predictions:
            1) binary classifier
            2) continuous regression
    Implementeted as a valid sklearn estimator, so it can be used in pipelines and GridSearch objects.
    Args:
        clf_name: currently supports either 'logistic' or 'LGBMClassifier'
        reg_name: currently supports either 'linear' or 'LGBMRegressor'
        clf_params: dict of parameters to pass to classifier sub-model when initialized
        reg_params: dict of parameters to pass to regression sub-model when initialized
    """

    def __init__(self,
                 clf_name: str = 'logistic', 
                 reg_name: str = 'linear',
                 clf_params: Optional[dict] = None,
                 reg_params: Optional[dict] = None):

        self.clf_name = clf_name
        self.reg_name = reg_name
        self.clf_params = clf_params
        self.reg_params = reg_params

    @staticmethod
    def _resolve_estimator(func_name: str):
        """ Lookup table for supported estimators.
        This is necessary because sklearn estimator default arguments
        must pass equality test, and instantiated sub-estimators are not equal. """

        funcs = {'linear': LinearRegression(),
                 'logistic': LogisticRegression(solver='liblinear'),
                 'LGBMRegressor': LGBMRegressor(n_estimators=50),
                 'LGBMClassifier': LGBMClassifier(n_estimators=50), 
                 'DTClassifier' : DecisionTreeClassifier(), 
                 'DTRegressor' : DecisionTreeRegressor(), 
                 'RFClassifier' :RandomForestClassifier(), 
                 'RFRegressor' : RandomForestRegressor(),
                 'GBClassifier' : GradientBoostingClassifier(), 
                 'GBRegressor' : GradientBoostingRegressor(), 
                }

        return funcs[func_name]

    def fit(self,
            X: Union[np.ndarray, pd.DataFrame],
            y: Union[np.ndarray, pd.Series]):
        X, y = check_X_y(X, y, dtype=None,
                         accept_sparse=False,
                         accept_large_sparse=False,
                         force_all_finite='allow-nan')

        if X.shape[1] < 2:
            raise ValueError('Cannot fit model when n_features = 1')

        self.clf_ = self._resolve_estimator(self.clf_name)
        if self.clf_params:
            self.clf_.set_params(**self.clf_params)
        self.clf_.fit(X, y > 0)

        self.reg_ = self._resolve_estimator(self.reg_name)
        if self.reg_params:
            self.reg_.set_params(**self.reg_params)
        self.reg_.fit(X[y > 0], y[y > 0])

        self.is_fitted_ = True
        return self

    def predict(self, X: Union[np.ndarray, pd.DataFrame]):
        """ Predict combined response using binary classification outcome """
        X = check_array(X, accept_sparse=False, accept_large_sparse=False)
        check_is_fitted(self, 'is_fitted_')
        return self.clf_.predict(X) * self.reg_.predict(X)

    def predict_expected_value(self, X: Union[np.ndarray, pd.DataFrame]):
        """ Predict combined response using probabilistic classification outcome """
        X = check_array(X, accept_sparse=False, accept_large_sparse=False)
        check_is_fitted(self, 'is_fitted_')
        return self.clf_.predict_proba(X)[:, 1] * self.reg_.predict(X)

'''
def manual_test():
    """ Validate estimator using sklearn's provided utility and ensure it can fit and predict on fake dataset. """
    check_estimator(HurdleRegression())
    from sklearn.datasets import make_regression
    X, y = make_regression()
    reg = HurdleRegression()
    reg.fit(X, y)
    reg.predict(X)


if __name__ == '__main__':
    manual_test()
'''

'\ndef manual_test():\n    """ Validate estimator using sklearn\'s provided utility and ensure it can fit and predict on fake dataset. """\n    check_estimator(HurdleRegression())\n    from sklearn.datasets import make_regression\n    X, y = make_regression()\n    reg = HurdleRegression()\n    reg.fit(X, y)\n    reg.predict(X)\n\n\nif __name__ == \'__main__\':\n    manual_test()\n'

# Classifier: Logistic regression, Regressor: Linear Regression
<b> The following models implement these 3 aspects in the following order
- Model creation
- Cross validation 
- Accuracy for 0s

In [33]:

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression



# split data into features and target
X_train = train_data.drop(columns=['retweets'])
y_train = train_data['retweets']
X_test = test_data.drop(columns=['retweets'])
y_test = test_data['retweets']

# initialize HurdleRegression model
clf_name = 'logistic'
reg_name = 'linear'

model = HurdleRegression(clf_name=clf_name, reg_name=reg_name)

# fit model on training data
model.fit(X_train, y_train)

# make predictions on test data
y_pred = model.predict(X_test)

# evaluate predictions using root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 30475.535875067104


In [18]:
#Getting a list of all the predictions 
preds = model.predict_expected_value(X_test)

In [19]:
print(preds)

[1.33154249 0.73193691 2.22962226 ... 0.15154977 6.88250507 0.11577636]


Cross validation for the first instance of the Hurdle model

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# define the scorer
scorer = make_scorer(mean_squared_error, squared=False)

# cross-validate the model using the scorer
scores = cross_val_score(model, X_train, y_train, cv=5, scoring=scorer)

# print the average RMSE and its standard deviation
print('Scores:', scores)
print("Average RMSE:", scores.mean())
print("Standard deviation:", scores.std())

Accuracy for zeros

In [ ]:
#Looking at the 0-1 accuracy
preds = model.predict_expected_value(X_test)
preds_int = pd.Series(preds.round().astype(int)).to_numpy()
sum(preds_int == y_test.to_numpy())/len(preds_int)

In [ ]:
sum(np.multiply((preds_int == y_test.to_numpy()), y_test.to_numpy() == 0))/sum(y_test.to_numpy() == 0)

#can use lasso penalty for logistic regresion hyperparameter tuning 


# Classifier: LGBM, Regressor: LGBM

In [46]:
#try boosted tree as a regressor 

# initialize HurdleRegression model
clf_name = 'LGBMClassifier'
reg_name = 'LGBMRegressor'

model_2 = HurdleRegression(clf_name=clf_name, reg_name=reg_name)

# fit model on training data
model_2.fit(X_train, y_train)

# make predictions on test data
y_pred = model_2.predict(X_test)

# evaluate predictions using root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 29.172875901019193


In [25]:
#Looking at the 0-1 accuracy
preds_2 = model_2.predict_expected_value(X_test)
preds_2_int = pd.Series(preds_2.round().astype(int)).to_numpy()
sum(preds_2_int == y_test.to_numpy())/len(preds_2_int)

0.5741812840231574

In [26]:
sum(np.multiply((preds_2_int == y_test.to_numpy()), y_test.to_numpy() == 0))/sum(y_test.to_numpy() == 0)

0.7834258863733448

In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error

# define the scorer
scorer = make_scorer(mean_squared_error, squared=False)

# cross-validate the model using the scorer
scores_2 = cross_val_score(model_2, X_train, y_train, cv=5, scoring=scorer)

# print the average RMSE and its standard deviation
print('Scores:', scores_2)
print("Average RMSE:", scores_2.mean())
print("Standard deviation:", scores_2.std())


Scores: [12.36066237 14.74897737 14.49042077 20.0200894  11.65086527]
Average RMSE: 14.654203038080755
Standard deviation: 2.9362871050708796


In [29]:
pip install shap

     -------------------------------------- 435.6/435.6 kB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [30]:
import shap
explainer = shap.TreeExplainer(model_2)
shap_values = explainer.shap_values(X_test)

InvalidModelError: Model type not yet supported by TreeExplainer: <class '__main__.HurdleRegression'>

In [ ]:
#can try boosted tree for classification as well 


In [ ]:
#need to use shap to 
#can make it a multi classification model - predict into retweet number groups 

# Decision trees for classifiers and regressors

In [35]:
# initialize HurdleRegression model
clf_name = 'DTClassifier'
reg_name = 'DTRegressor'

model_3 = HurdleRegression(clf_name=clf_name, reg_name=reg_name)

# fit model on training data
model_3.fit(X_train, y_train)

# make predictions on test data
y_pred = model_3.predict(X_test)

# evaluate predictions using root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 30.428483538195728


In [48]:
# cross-validate the model using the scorer
scores_3 = cross_val_score(model_3, X_train, y_train, cv=5, scoring=scorer)

# print the average RMSE and its standard deviation
print('Scores:', scores_3)
print("Average RMSE:", scores_3.mean())
print("Standard deviation:", scores_3.std())


Scores: [15.97209252 22.26093198 17.33156294 20.08344546 17.38698044]
Average RMSE: 18.607002668606015
Standard deviation: 2.2622570565544335


In [ ]:
#Looking at the 0-1 accuracy
preds_3 = model_3.predict_expected_value(X_test)
preds_3_int = pd.Series(preds_3.round().astype(int)).to_numpy()
sum(preds_3_int == y_test.to_numpy())/len(preds_3_int)

In [ ]:
sum(np.multiply((preds_3_int == y_test.to_numpy()), y_test.to_numpy() == 0))/sum(y_test.to_numpy() == 0)

# Random forests for classifiers and regressors

In [38]:
# initialize HurdleRegression model
clf_name = 'RFClassifier'
reg_name = 'RFRegressor'

model_4 = HurdleRegression(clf_name=clf_name, reg_name=reg_name)

# fit model on training data
model_4.fit(X_train, y_train)

# make predictions on test data
y_pred = model_4.predict(X_test)

# evaluate predictions using root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 26.87993552144905


In [ ]:
# cross-validate the model using the scorer
scores_4 = cross_val_score(model_4, X_train, y_train, cv=5, scoring=scorer)

# print the average RMSE and its standard deviation
print('Scores:', scores_4)
print("Average RMSE:", scores_4.mean())
print("Standard deviation:", scores_4.std())


In [ ]:
#Looking at the 0-1 accuracy
preds_4 = model_3.predict_expected_value(X_test)
preds_4_int = pd.Series(preds_4.round().astype(int)).to_numpy()
sum(preds_4_int == y_test.to_numpy())/len(preds_4_int)

In [ ]:
sum(np.multiply((preds_4_int == y_test.to_numpy()), y_test.to_numpy() == 0))/sum(y_test.to_numpy() == 0)

# Gradient boosting for classifiers and regressors

In [39]:
# initialize HurdleRegression model
clf_name = 'GBClassifier'
reg_name = 'GBRegressor'

model_5 = HurdleRegression(clf_name=clf_name, reg_name=reg_name)

# fit model on training data
model_5.fit(X_train, y_train)

# make predictions on test data
y_pred = model_5.predict(X_test)

# evaluate predictions using root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

RMSE: 25.734126093120445


In [ ]:
# cross-validate the model using the scorer
scores_5 = cross_val_score(model_5, X_train, y_train, cv=5, scoring=scorer)

# print the average RMSE and its standard deviation
print('Scores:', scores_5)
print("Average RMSE:", scores_5.mean())
print("Standard deviation:", scores_5.std())


In [ ]:
#Looking at the 0-1 accuracy
preds_5 = model_3.predict_expected_value(X_test)
preds_5_int = pd.Series(preds_5.round().astype(int)).to_numpy()
sum(preds_5_int == y_test.to_numpy())/len(preds_5_int)

In [ ]:
sum(np.multiply((preds_4_int == y_test.to_numpy()), y_test.to_numpy() == 0))/sum(y_test.to_numpy() == 0)